In [ ]:
import torchaudio
import torchaudio.sox_effects as sox
import numpy as np
from IPython.display import Audio

In [ ]:
sox.init_sox_effects()

In [ ]:
sox.effect_names()

In [ ]:
#Effect params
from torchaudio.utils.sox_utils import list_effects
list_effects()

In [ ]:
waveform, sample_rate = torchaudio.load('../datasets/GTZAN/gtzan_genre/genres/disco/disco.00013.wav')
waveform = waveform.mean(dim=0, keepdim=True)  # convert stereo to mono
Audio(waveform, rate=sample_rate)

In [ ]:
# Define the effect parameters using numpy
gain = np.random.randint(-12, 0)
pitch = np.random.randint(-1200, 1200)
reverb_params = [np.random.randint(0, 100)] * 3
chorus_params = [
    round(np.random.uniform(0.1, 1.0), 1),
    round(np.random.uniform(0.1, 1.0), 1),
    55,
    round(np.random.uniform(0.1, 0.9), 1),
    round(np.random.uniform(0.1, 2.0), 2),
    np.random.randint(2, 5),
    np.random.choice(["-s", "-t"]),
]
drive = np.random.randint(0, 30)
stretch = round(np.random.uniform(0.8, 1.2), 1)
speed = np.random.uniform(0.7, 1.3)
tremolo_speed = np.random.uniform(0.1, 100)
tremolo_depth = np.random.randint(1, 101)

# Define the effect chain using f-strings
effects = [
    ["gain", "-n", f"{gain}"],
    ["chorus", *map(str, chorus_params)],
    ["overdrive", f"{drive}"],
    ["pitch", f"{pitch}"],
    ["reverb", *[str(param) for param in reverb_params]],
    ["speed", f"{speed}"],
    ["stretch", f"{stretch}"],
    ["tremolo", f"{tremolo_speed}", f"{tremolo_depth}"],
]

positive, _ = sox.apply_effects_tensor(waveform, sample_rate, effects)
positive = positive.mean(dim=0, keepdim=True)  # convert stereo to mono

In [ ]:
positive, sample_rate = sox.apply_effects_tensor(waveform, sample_rate, effects)
#torchaudio.save('positive.wav', positive, sample_rate)
Audio(waveform, rate=sample_rate)